In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from parsel import Selector
from openpyxl import Workbook

# Inicialização do driver do Firefox
driver = webdriver.Firefox()

# Abrindo a página
url = 'https://www.google.com/maps/place/Cl%C3%ADnica+Freire+Odontologia/@-21.0115254,-40.8114232,15z/data=!4m8!3m7!1s0xb93b83074dcca3:0xe7f1f42dee0de2bc!8m2!3d-21.0115254!4d-40.8114232!9m1!1b1!16s%2Fg%2F11fp8l28qv?entry=ttu'
driver.get(url)

# Esperar até que pelo menos uma avaliação seja carregada na página
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH, '//div[@data-review-id]')))

# Obter o conteúdo da página e criar um objeto Selector
page_content = driver.page_source
response = Selector(text=page_content)

# Extrair os dados das avaliações
results = []

for el in response.xpath('//div[@data-review-id]'):
    body = el.xpath('.//span[contains(@class, "section-review-text")]/text()').get(default='')
    time = el.xpath('.//span[contains(@class, "section-review-publish-date")]/@aria-label').get(default='')
    results.append({
        'body': body,
        'time': time
    })

# Fechar o navegador
driver.quit()

# Escrever os resultados em uma planilha Excel
wb = Workbook()
ws = wb.active
ws.append(['Body', 'Time'])

for result in results:
    ws.append([result['body'], result['time']])

wb.save('reviews.xlsx')
